In [160]:
from datetime import datetime
import sys
from SPARQLWrapper import SPARQLWrapper, JSON
from PIL import Image
from markdownify import markdownify 
import folium
import io




# endpoint_url = "https://computational-publishing-service.wikibase.cloud/query/sparql"

endpoint_url = "https://query.wikidata.org/sparql"

query = """
#defaultView:Map

SELECT DISTINCT ?item ?itemLabel ?location ?locationLabel ?latitude ?longitude ?geo WHERE {
  VALUES ?type { wd:Q1007870 wd:Q33506 wd:Q166118 wd:Q2668072 wd:Q667276 
                 wd:Q464980 wd:Q56190453 }
  #art gallery, museum, archive, collection, art exhibition, 
  # exhibition, memorial
  
  VALUES ?places { wd:Q1715 } 

  # Find locations that match ?places directly OR one level above/below
  ?place (wdt:P131|^wdt:P131) ?places. 

  # Find items that are in those locations
  ?item wdt:P31 ?type;  # Must be an instance of a relevant cultural type
        wdt:P276|wdt:P131 ?place.  # Located in one of the places
  
  ?item wdt:P625 ?geo.  # Get coordinates

  SERVICE wikibase:label { bd:serviceParam wikibase:language "en,[AUTO_LANGUAGE]". }

  BIND(geof:latitude(?geo) AS ?latitude).
  BIND(geof:longitude(?geo) AS ?longitude).
}
"""



def run_query(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setMethod("POST") #this NEEDS to be added to get results (not included in the wikibase python example code)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

locations = []

def get_items():

#    print("<link rel='stylesheet' href='./masonrydesign.css'>")
    q = query

    results = run_query(endpoint_url, q)

    for item in results["results"]["bindings"]:    

      alt='image of a ' + item['itemDescription']['value'] if 'itemDescription' in item else 'image of a brutalist building'
      title=item['itemLabel']['value'] if 'itemLabel' in item else ''
      year=item['year']['value'] if 'year' in item else ''
      latitude=item['latitude']['value'] if 'latitude' in item else ''
      longitude=item['longitude']['value'] if 'longitude' in item else ''
      place=item['placeLabel']['value'] + ', ' + item['countryLabel']['value'] if 'placeLabel' in item else ''
      item_url=item['item']['value'] if 'item' in item else ''
      image_url=item['image']['value'] if 'image' in item else ''

      locations.append((title, latitude, longitude, image_url, item_url, place, alt))







In [161]:
get_items()

m = folium.Map(location=[52.38, 9.74], zoom_start=12)

for name, lat, lon, url, item, place, alt in locations:
    folium.Marker(
        location=[lat, lon],
        popup=f"""<h3><a href='{item}' target='_blank'>{name}</a></h3>
        {place}""",
        #<img src='{url}' alt='{alt}' style='width:200px;height:200px;object-fit:contain;'>
        #<a href='{url}' target='_blank'>Full size image</a>""",  
        tooltip=name  # Hover label
    ).add_to(m)

print(""" 
::: {.content-visible when-format='html'}
""")   



 
::: {.content-visible when-format='html'}



In [162]:

m



In [163]:
print("""
:::
""")



:::



In [164]:

# Generate image from HTML
img_data = m._to_png(5)
img = Image.open(io.BytesIO(img_data))

img.save('images/museumsmap.png')

print(""" 
:::{.content-visible unless-format='html'}
      
![](images/museumsmap.png)

:::
""")


 
:::{.content-visible unless-format='html'}
      
![](images/museumsmap.png)

:::



In [165]:
#get_graph()